## 모델 제작 순서에 따른 이론 한눈에 보기
---


#### 0. 라이브러리 호출
---

In [3]:
import torch
import torch.nn as nn
import torch.autograd 
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms # 데이터 전처리를 위해 사용하는 라이브러리
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt
import numpy as np
import time

#### 1. device 설정
---

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #  torch.cuda.is_available() GPU를 사용가능하면 True, 아니라면 False를 리턴

print("지금 사용하는 device :",device)

지금 사용하는 device : cuda:0


#### 2. 데이터셋 준비
---

#### 3. 신경망 생성
---

보통 출력층으로 회귀일 경우 항등함수, 분류일 경우 소프트맥스 함수로 해둔다

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

#### 4. 모델 설정 완료
---

1. 신경망을 모델로 설정하기

2. 손실함수 설정 (criterion)
    - 오차 제곱합, 교차 엔트로피 오차 등등...
    </br></br>

3. 옵티마이저 설정 (optimizer)
    신경망 학습의 목적은 손실 함수의 값을 가능한 한 낮추는 매개변수를 찾는 것이다. 이는 곧 매개변수의 최적값을 찾는 문제이며, 이러한 문제를 푸는 것이 최적화(optimization)이라 한다. 

    최적화 기법은 여러가지가 있다. (SGD, 모멘텀, Adam 등등..) 이러한 최적화를 해주는 클래스가 옵티마이저(Optimizer)이다. 옵티마이저를 한국어로 해석하면 '최적화를 행하는 자'라는 뜻이다.

4. 경사법 설정 이론
    - 경사법 : 보통 그레디언트가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향이다. 우리는 손실함수를 가지고 모델의 정확도를 평가한다.

        ![1](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_13.png)

        입력 데이터(x)를 넣어서 손실 함수(y)가 나왔으니, 손실 함수에서 역전파를 흘려주면 

        ![2](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_15.png)

        손실 함수에 대한 기울기가 나오게 된다. (dx/dy)

        ![3](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_14.png)
    
        따라서 손실 함수를 줄이는 방향으로 경사법을 사용해야 한다. 

        신경망이 최적의 매개변수를 가지고 있다면? 손실 함수가 줄어들것이다. 즉, 최적이란 손실 함수가 최솟값이 될 때의 매개변수 값이다. 
        
        그러나 일반적인 문제의 손실 함수는 매우 복잡하다. 매개변수 공간이 광대하여 어디가 최솟값이 되는 곳인지를 짐작할 수 없다.

        이런 상황에서 기울기를 잘 이용해 함수의 최솟값(또는 가능한 한 작은 값)을 찾으려는 것이 경사법이다.

        > 경사법 : 현 위치에서 기울어진 방향으로 일정 거리만큼 이동한다. 그런 다음 이동한 곳에서도 기울기를 구한 뒤 또 기울어진 방향으로 나아간다 이렇게 해서 함수의 값을 줄이는 것이 경사법이다

        ![4](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_16.png)

        ```param.data -= self.lr * param.grad.data```

        보통의 파라미터(매개변수)는 이렇게 업데이트가 된다. 

        ![5](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_17.jpg)

        이렇게 모든 계층을 돌면서 각 계층마다 입력값을 줄이는 벡터(기울기)가 한번에 구해진다!
        
        입력값을 줄이는 것은 알겠는데 그래서 파라미터는 어떻게 줄이나요? 라고 한다면 계산 그래프에 대한 정의를 다시 생각해보자

        ![6](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_18.png)

        위는 완전연결 계층의 계산 그래프이다. 계산 그래프의 입력값에 가중치(매개변수)가 있음을 까먹지 말라! 

        이쁘게 정리하면 다음과 같다.

        1. 입력에 따른 손실 함수를 구한다.
        
        2. 손실 함수를 줄이는 방법으로 경사법을 사용하자

        3. 이때 손실 함수까지 형성된 계산 그래프를 역전파 시키면 손실 함수(dL/dL)를 줄이기 위한 각 매개변수(가중치일 경우 dL/dW)의 기울기가 나온다!!

        4. 나온 기울기를 바탕으로 경사법을 이용해 손실 함수를 줄인다

        ![7](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_19.jpg)
        

    


#### 5. 모델 학습하는 함수 만들기
---


In [1]:
def train_model(model, dataloaders, crtierion, optimizer, device, save_path, num_epochs=5):
    since = time.time()
    acc_history = [] # 각 에폭마다 정확도 저장
    loss_history = [] # 각 에폭마다 손실함수값 저장
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        running_loss = 0.0 # 손실 함수
        running_corrects = 0 # 정답갯수

        for inputs, labels in dataloaders:
            inputs.requires_grad_(True)
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.to(device)

            outputs = model(inputs) # 순전파
            loss = crtierion(outputs, labels) # 손실 함수 구하기
            optimizer.zero_grad() # 기울기를 0으로 설정
            _, preds = torch.max(outputs, 1) # 결과값 추출
            loss.backward() # 역전파 실행 이때 requires_grad = True가 된 완전연결층만 역전파가 됨
            optimizer.step() # 기울기 업데이트

            running_loss += loss.item() * inputs.size(0) # 출력 결과와 레이블의 오차를 계산한 결과를 누적하여 저장한다
            # loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있습니다.
            running_corrects += torch.sum(preds == labels.data) # 출력 결과와 레이블이 동일한지 확인한 결과를 누적하여 저장

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss,epoch_acc)) # 참고) {:.소수점 자리수f} 포맷 코드 : {} 내에 실수의 소수점 자리수(.소수점 자리수f)를 지정해 줄 수 있음 소수점 4자리 까지 표시함
        if epoch_acc > best_acc: # 만약에 어떤 에폭에서의 정확도가 최고 정확도보다 높을 경우 업데이트
            best_acc = epoch_acc
        
        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since # 실행 시간 계산
    print(f'실행 시간 : {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'이 모델의 최고 정확도: {best_acc:4f}') # 최고 정확도

    torch.save(model.state_dict(),save_path) # 모든 에폭을 소진할 때 모델의 상태를 저장 

    return acc_history, loss_history



#### 6. 모델 평가하는 함수 만들기
---

In [ ]:
def test_model(model,dataloaders,device,save_path):
    since = time.time()
    acc_history = []
    best_acc = 0.0

    model.load_state_dict(torch.load(save_path))
    model.eval() # 모델을 train에서 evaluation으로 변경 https://bluehorn07.github.io/2021/02/27/model-eval-and-train.html
    model.to(device)
    running_corrects = 0

    for inputs, labels in dataloaders:
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.no_grad(): # 자동미분을 사용하지 않음 (학습 목적이 아닌 테스트 목적)
            outputs = model(inputs)

            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        print('Acc: {:.4f}'.format(epoch_acc))

        if epoch_acc > best_acc: # 만약에 어떤 에폭에서의 정확도가 최고 정확도보다 높을 경우 업데이트
            best_acc = epoch_acc
        
        acc_history.append(epoch_acc.item())
        print()
    
    time_elapsed = time.time() - since # 실행 시간 계산
    print(f'Validation complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}') # 최고 정확도

    return acc_history